In [1]:
import numpy as np
import pandas as pd
import powerlaw
import networkx as nx
from netanalytics.power_law_testing import monte_carlo_resampling, p_value

In [2]:
p_values_state = []
mean_p_values_RW = []
std_p_values_RW = []

k = 0
while k<5:
    print('Iteration {}'.format(k))
    xmin = np.random.randint(2,100)
    alpha = np.random.choice(np.linspace(2,3,40))
    distr = powerlaw.Power_Law(xmin = xmin, parameters = [alpha], discrete = True)
    data = distr.generate_random(100000,estimate_discrete=True)
    if np.sum(data)%2!=0:
        data[np.where(data==np.min(data))[0][0]] = data[np.where(data==np.min(data))[0][0]]+1
    
    
    noise = [np.random.choice(np.unique(data),n).astype(int) for n in range(10,110,30)]
    noisy_data = [np.hstack((data,noise[i])) for i in range(len(noise))]
    for i in range(len(noisy_data)):
        if np.sum(noisy_data[i])%2!=0:
            noisy_data[i][np.where(noisy_data[i]==np.min(noisy_data[i]))[0][0]] = noisy_data[i][np.where(noisy_data[i]==np.min(noisy_data[i]))[0][0]]+1
    
    
    data = data.astype(int)
    p_values_state.append(p_value(data))

    for i in range(len(noisy_data)):
        p_values_state.append(p_value(noisy_data[i]))
    
    
    
    #G = nx.configuration_model(data, create_using=nx.Graph())
    resamplings = monte_carlo_resampling(int(len(data)/2),data,5)
    p_values_MC = [p_value(resamplings[i], 25)
                   for i in range(len(resamplings))]
    mean_p_values_MC.append(np.mean(p_values_MC))
    std_p_values_MC.append(np.std(p_values_MC))
    
    for j in range(len(noisy_data)):
        print("Amount of noise: {} noisy observations".format(len(noisy_data[j])-100000))
        noisy_data[j] = noisy_data[j].astype(int)
        #G = nx.configuration_model(noisy_data[j], create_using=nx.Graph())


        resamplings = MC_undirected_sequence(int(len(noisy_data[j])/2),noisy_data[j],5)

        p_values_MC = [p_value(25,resamplings[i]) for i in range(len(resamplings))]
        p_values_MC = np.array(p_values_MC)
        mean_p_values_MC.append(np.mean(p_values_MC))
        std_p_values_MC.append(np.std(p_values_MC))

    k = k+1

Iteration 0


Calculating best minimal value for power law fit
/home/vero/miniconda2/envs/py36/lib/python3.6/site-packages/powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/home/vero/miniconda2/envs/py36/lib/python3.6/site-packages/powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
p_values_state = np.array(p_values_state).reshape((5,5)).T

In [ ]:
mean_p_values_MC = np.array(mean_p_values_MC).reshape((5,5)).T

In [ ]:
box = np.zeros((10,5))
box[np.arange(0,10,2),:]=p_values_state
box[np.arange(1,10,2),:]=mean_p_values_MC

In [ ]:
from matplotlib import rcParams
from matplotlib.lines import Line2D

rcParams.update({'font.size':18})
plt.figure(figsize=(15,4))
bp = plt.boxplot(boxplots,  positions=[1.20, 1.80, 3.20, 3.80, 5.20, 5.80, 7.20, 7.80,9.20, 9.80], #notch=True,  # notch shape
                         vert=True,patch_artist=True, medianprops=dict(color='red', linewidth=3))
plt.setp(bp['boxes'], color='black')
plt.setp(bp['whiskers'], color='black')
plt.setp(bp['fliers'], color='red', marker='+')
colors = ['pink', 'lightblue']
# for bplot in (bplot_):
#     for patch, color in zip(bplot['boxes'], colors):
#         patch.set_facecolor(color)
num_boxes = 10
medians = np.empty(num_boxes)
for i in range(num_boxes):
    box = bp['boxes'][i]
    box.set_facecolor(colors[i%2])

    
custom_lines = [Line2D([0], [0], color='pink', lw=5),
                Line2D([0], [0], color='lightblue', lw=5),
                Line2D([0], [0], color='red', lw=1, linestyle='--'),
               ]

plt.legend(custom_lines, ['Bootstrap', 'Monte Carlo + Bootstrap', 'Significance level'], ncol=3, 
          bbox_to_anchor=(0.86, 1.2))
plt.axhline(0.1, color='red', linestyle='--')
plt.xticks(np.arange(1.5, 10, 2), [0, 10, 40, 70, 100])
plt.xlabel('Number of noisy added observations')
plt.ylabel(r'$p$-value')
# plt.savefig("incrementing_noise.pdf", dpi=200, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
powerlaw.plot_pdf(noisy_data[3])

In [ ]:
np.save('box',box)

In [ ]:
list(range(10,110,30))